In [1]:
import tensorflow as tf
import numpy as np

backend = tf.keras.backend
models = tf.keras.models
layers = tf.keras.layers

In [4]:
class Dilated_Block():
    def __init__(self, filters, kernel_size, rate = 1):
        super(dilated_block, self).__init__()
        
        self.bn = layers.BatchNormalization()
        self.relu = layers.Activation('relu')
        self.conv = layers.Conv2D(filters, kernel_size, padding = 'same', dilation_rate = rate)
        
    def call(self, inputs):
        
        x = self.bn(x)
        x = self.relu(x)
        x = self.conv(x)
        
        return x

In [5]:
class Modified_ASPP():
    def __init__(self, num_classes):
        super(Modified_ASPP, self).__init__()
        
        self.conv1x1 = layers.Conv2D(256, 1, padding = 'same')
        self.conv17x17 = layers.Conv2D(64, 17, padding = 'same')
        self.conv13x13 = layers.Conv2D(64, 13, padding = 'same')
        self.conv9x9 = layers.Conv2D(64, 9, padding = 'same')
        self.conv5x5 = layers.Conv2D(64, 5, padding = 'same')
        
        self.conv3 = layers.Conv2D(64, 3, padding = 'same')
        self.conv5 = layers.Conv2D(64, 5, padding = 'same')
        self.conv7 = layers.Conv2D(64, 7, padding = 'same')
        
        self.mpooling1 = layers.MaxPooling2D(size = (2, 2), strides = (1, 1), padding = 'same')
        self.mpooling2 = layers.MaxPooling2D(size = (2, 2), strides = (1, 1), padding = 'same')
        self.mpooling3 = layers.MaxPooling2D(size = (2, 2), strides = (1, 1), padding = 'same')
        self.mpooling4 = layers.MaxPooling2D(size = (2, 2), strides = (1, 1), padding = 'same')
        
        self.dilated1 = Dilated_Block(256, 1)
        self.dilated2 = Dilated_Block(64, 3, rate = 3)
        
        self.dilated3 = Dilated_Block(256, 1)
        self.dilated4 = Dilated_Block(64, 3, rate = 6)
        
        self.dilated5 = Dilated_Block(256, 1)
        self.dilated6 = Dilated_Block(64, 3, rate = 12)
        
        self.upsam1 = layers.UpSampling2D(size = (4, 4))
        self.upsam2 = layers.UpSampling2D(size = (4, 4))
        
        self.outputs = layers.Conv2D(num_classes, 1, strides = 1)
        
        
        
    def call(self, inputs):
        
        # 1x1 convolution layer
        inpts = self.conv1x1(inputs)
        
        # conv 17, 13, 9, 5
        x_17 = self.conv17x17(inpts)
        x_13 = self.conv13x13(inpts)
        x_9 = self.conv9x9(inpts)
        x_5 = self.conv5x5(inpts)
        
        # conv 3, 5, 7
        x3 = self.conv3(inpts)
        x5 = self.conv5(inpts)
        x7 = self.conv7(inpts)
        
        dil3 = self.dilated1(inpts)
        dil3 = self.dilated2(dil3)
        
        n_inputs1 = backend.concatenate([inpts, dil3])
        
        dil6 = self.dilated3(n_inputs1)
        dil6 = self.dilated4(dil6)
        
        n_inputs2 = backend.concatenate([n_inputs1, dil6])
        
        dil12 = self.dilated5(n_inputs2)
        dil12 = self.dilated6(dil12)
        
        last_concat = backend.concatenate([inpts, x_17, x_13, x_9, x_5, x3, x5, x7, dil3, dil6, dil12])
        
        x = self.outputs(last_concat)
        x = self.upsam1(x)
        x = self.upsam2(x)
        
        return x